In [1]:
from stable_baselines3 import PPO
import gymnasium as gym
from env import Env2048
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.utils import get_action_masks
from sb3_contrib import MaskablePPO
import numpy as np
import os

/opt/anaconda3/envs/MasterThesis/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
gym.register(
    id="gymnasium_env/Env2048",
    entry_point=Env2048,
)

In [3]:
vec_env = make_vec_env("gymnasium_env/Env2048", n_envs=4, env_kwargs={"render_mode": "human", "max_box_number": [512]})


In [ ]:
use_pretrained = True

if os.path.exists("maskable_ppo_2048.zip") and use_pretrained:
    model = MaskablePPO.load("maskable_ppo_2048")
    model.set_env(vec_env)
else:
    model = MaskablePPO(
        "MlpPolicy",
        vec_env,
        verbose=1,
        ent_coef=0.01,
        learning_rate=5e-4,
        clip_range=0.2,
        vf_coef=0.7,
        target_kl=0.03,
        tensorboard_log="./ppo_mask_2048_tensorboard/"
    )
model.learn(total_timesteps=300000)
model.save("maskable_ppo_2048")
del model

Using cpu device
Logging to ./ppo_mask_2048_tensorboard/PPO_1


/opt/anaconda3/envs/MasterThesis/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/anaconda3/envs/MasterThesis/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 120      |
|    ep_rew_mean     | 1.01e+03 |
| time/              |          |
|    fps             | 2731     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 119         |
|    ep_rew_mean          | 982         |
| time/                   |             |
|    fps                  | 1911        |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.008159927 |
|    clip_fraction        | 0.0564      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.23       |
|    explained_variance   | -8.69e-05   |
|    learning_rate        | 0.

In [8]:
if os.path.exists("ppo_2048.zip"):
    model = PPO.load("ppo_2048")
    model.set_env(vec_env)
else:
    model = PPO(
        "MlpPolicy",
        vec_env,
        verbose=1,
        ent_coef=0.005,
        learning_rate=0.0003,
        vf_coef=0.7,
        target_kl=0.03,
        tensorboard_log="./ppo_2048_tensorboard/",
    )
model.learn(total_timesteps=500000)
model.save("ppo_2048")
del model

Logging to ./ppo_2048_tensorboard/PPO_12
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 126      |
|    ep_rew_mean     | 880      |
| time/              |          |
|    fps             | 1549     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 8192     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 126           |
|    ep_rew_mean          | 881           |
| time/                   |               |
|    fps                  | 473           |
|    iterations           | 2             |
|    time_elapsed         | 34            |
|    total_timesteps      | 16384         |
| train/                  |               |
|    approx_kl            | 0.00043469464 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.22         |
|  

In [1]:
%load_ext tensorboard
%tensorboard --logdir ppo_mask_2048_tensorboard

In [4]:
model = PPO.load("ppo_2048")

In [12]:
env = gym.make("gymnasium_env/Env2048", render_mode="human", max_box_number=[1024])

In [13]:
model = MaskablePPO.load("maskable_ppo_2048")

In [8]:
env.reset()
get_action_masks(env)

array([ True,  True,  True,  True])

In [14]:
obs, _= env.reset()
done = False
games = 0
try:
    env.render()
    while True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(obs, action_masks=action_masks, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        env.render()

        # Se l'episodio è terminato, resetta
        if done:
            obs, _ = env.reset()
            games += 1

        if games >= 3:
            break
    print("Episodi completati:", games)

except KeyboardInterrupt:
    print("Interrotto dall'utente")
finally:
    env.close()

Episodi completati: 3


In [6]:
obs, _= env.reset()
done = False
games = 0
try:
    env.render()
    while True:
        action, _states = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        env.render()
        
        # Se l'episodio è terminato, resetta
        if done:
            obs, _ = env.reset()
            games += 1

        if games >= 3:
            break
    print("Episodi completati:", games)
            
except KeyboardInterrupt:
    print("Interrotto dall'utente")
finally:
    env.close()

Interrotto dall'utente


In [ ]:
env.close()

: 